### Import Stuff

In [8]:
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
import pyspark.sql.types as T

### Create SQL Context

In [9]:
sqlContext = SQLContext(sc)

### Create Dataframes on Cassandra tables

In [10]:
stores_df = sqlContext.read\
    .format("org.apache.spark.sql.cassandra")\
    .options(keyspace="retail", table="stores")\
    .load()

In [11]:
receipts_by_store_date_df = sqlContext.read\
    .format("org.apache.spark.sql.cassandra")\
    .options(keyspace="retail", table="receipts_by_store_date")\
    .load()

### Create UDFs - These may not perform well, but are run only on the final results

In [12]:
concat = udf(lambda s1, s2: s1 + s2, T.StringType())

### Compute the sales by state
1. join receipts_by_store_date to store
2. group by state
3. sum by receipt_total
4. do a select to add the dummy column, rename columns, compute the region and round the totals

In [13]:
sales_by_state_df = receipts_by_store_date_df\
   .join(stores_df, stores_df.store_id == receipts_by_store_date_df.store_id )\
   .groupBy("state")\
   .sum("receipt_total")\
   .select(F.lit("dummy").alias("dummy"), "state", concat(F.lit("US-"),"state").alias("region"), F.col("SUM(receipt_total)").cast("Decimal(10,2)").alias("receipts_total"))

### Save it

In [14]:
sales_by_state_df.write\
      .format("org.apache.spark.sql.cassandra")\
      .options(keyspace = "retail", table = "sales_by_state")\
      .save()

Py4JJavaError: An error occurred while calling o180.save.
: java.lang.UnsupportedOperationException: 'Writing to a non-empty Cassandra Table is not allowed.'
	at org.apache.spark.sql.cassandra.DefaultSource.createRelation(DefaultSource.scala:93)
	at org.apache.spark.sql.sources.ResolvedDataSource$.apply(ddl.scala:309)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:144)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:497)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:231)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:379)
	at py4j.Gateway.invoke(Gateway.java:259)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j